In [10]:
import csv
import math
import os
import cv2
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageEnhance
import numpy as np
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_ir
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mv
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape, DepthwiseConv2D, ZeroPadding2D, Dense, Average, Lambda, Input, Concatenate
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon
from tensorflow.keras import Sequential

from sklearn.model_selection import train_test_split

DATA_DIR = "images"

# 0.35, 0.5, 0.75, 1.0, 1.3, 1.4
ALPHA = 1.3

# 96, 128, 160, 192, 224
IMAGE_SIZE = 299

EPOCHS = 200
BATCH_SIZE = 24
PATIENCE = 50

MULTI_PROCESSING = True
THREADS = 12

data = pd.read_csv('training.csv')
train, test = train_test_split(data, test_size=0.1, random_state=123)

train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train.to_csv('train.csv', index=False)
test.to_csv('validation.csv', index=False)

TRAIN_CSV = "train.csv"
VALIDATION_CSV = "validation.csv"

In [2]:
def create_irv2(trainable=False):
    model = InceptionResNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(512, kernel_size=4, name="coords1", activation='relu')(x)
    x = Conv2D(256, kernel_size=3, name="coords2", activation='relu')(x)
    x = Conv2D(4, kernel_size=3, name="coords3", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)

def create_mnv2(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(256, kernel_size=3, name="coords1", activation='relu')(x)
    x = Conv2D(4, kernel_size=5, name="coords2", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)


In [3]:
irv2 = create_irv2()
irv2.load_weights('inception_resnetV2-no_weights-freeze-0.91.h5')


In [20]:
mnv2 = create_mnv2()
mnv2.load_weights('mobilenetV2-no-weights-0.91.h5')


In [5]:
train_df = pd.read_csv("training.csv")
train_df.head()

,image_name,x1,x2,y1,y2
0,JPEG_20160706_121146_1000145715002.png,115,495,143,325
1,JPEG_20161119_174038_1000690577600.png,23,457,61,409
2,147444927651111470309333776-Roadster-Men-Casua...,37,601,13,470
3,147772332675720161028_161611.png,27,602,162,385
4,1473315333651DeeplearnS11638.png,60,586,174,325


In [ ]:
df = pd.DataFrame(np.zeros((len(train_df), 13)))

In [13]:
for index, row in train_df.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_mv(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    df.iloc[index,0] = row['image_name']
    df.iloc[index,1] = ((x1a))
    df.iloc[index,2] = ((x2a))
    df.iloc[index,3] = ((y1a))
    df.iloc[index,4] = ((y2a))
    df.iloc[index,5] = ((x1b))
    df.iloc[index,6] = ((x2b))
    df.iloc[index,7] = ((y1b))
    df.iloc[index,8] = ((y2b))
    df.iloc[index,9] = row['x1']
    df.iloc[index,10] = row['x2']
    df.iloc[index,11] = row['y1']
    df.iloc[index,12] = row['y2']
    
    if (index%1000)==0:
        print(index)

df.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160706_121146_1000145715002.png,112.353850,485.023091,142.554674,322.287867,111.899054,490.973990,143.549995,318.539690,115.0,495.0,143.0,325.0
1,JPEG_20161119_174038_1000690577600.png,20.340435,461.615901,57.541990,407.892748,14.900318,458.294285,57.182395,411.727916,23.0,457.0,61.0,409.0
2,147444927651111470309333776-Roadster-Men-Casua...,31.943174,584.442282,12.664641,471.151119,38.461993,584.657026,12.636195,458.999241,37.0,601.0,13.0,470.0
3,147772332675720161028_161611.png,24.045902,602.235838,161.485551,375.219506,27.214124,598.370536,164.150211,382.583902,27.0,602.0,162.0,385.0
4,1473315333651DeeplearnS11638.png,63.669017,585.924044,173.707026,327.727680,59.576765,587.784816,173.057213,328.764561,60.0,586.0,174.0,325.0


In [14]:
df.to_csv('training2.csv')

In [15]:
val_df = pd.read_csv("validation.csv")
val_df.head()

,image_name,x1,x2,y1,y2
0,JPEG_20160624_135055_1000527160000.png,132,606,165,325
1,147771890839520161028_141911.png,123,544,52,452
2,1474715198275DSC07675.png,311,458,32,459
3,JPEG_20161117_112207_1000880661419.png,13,640,5,459
4,147444941744811473248953580-Roadster-Men-Navy-...,34,603,16,471


In [16]:
df2 =  pd.DataFrame(np.zeros((len(val_df), 13)))

In [17]:
for index, row in val_df.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_mv(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    df2.iloc[index,0] = row['image_name']
    df2.iloc[index,1] = ((x1a))
    df2.iloc[index,2] = ((x2a))
    df2.iloc[index,3] = ((y1a))
    df2.iloc[index,4] = ((y2a))
    df2.iloc[index,5] = ((x1b))
    df2.iloc[index,6] = ((x2b))
    df2.iloc[index,7] = ((y1b))
    df2.iloc[index,8] = ((y2b))
    df2.iloc[index,9] = row['x1']
    df2.iloc[index,10] = row['x2']
    df2.iloc[index,11] = row['y1']
    df2.iloc[index,12] = row['y2']
    
    if (index%1000)==0:
        print(index)

df2.head()

0
1000


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160624_135055_1000527160000.png,136.813655,593.797358,171.022002,318.161760,140.286015,600.110038,173.996577,317.819759,132.0,606.0,165.0,325.0
1,147771890839520161028_141911.png,123.851812,544.165611,62.043580,458.196952,123.266852,543.815613,60.365218,453.204313,123.0,544.0,52.0,452.0
2,1474715198275DSC07675.png,309.060769,461.285013,28.725136,455.320322,304.909690,457.154105,31.033978,463.411200,311.0,458.0,32.0,459.0
3,JPEG_20161117_112207_1000880661419.png,24.905832,643.299801,0.000000,477.573748,0.000000,630.595921,0.000000,480.802100,13.0,640.0,5.0,459.0
4,147444941744811473248953580-Roadster-Men-Navy-...,36.902999,601.593201,25.066516,471.227056,36.505154,601.419939,20.901620,471.695088,34.0,603.0,16.0,471.0


In [ ]:
df2.to_csv('validation2.csv')

In [6]:
etrain = pd.read_csv('training2.csv', index_col=0)
etrain.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160706_121146_1000145715002.png,114.592040,485.142500,145.374205,325.247488,111.899054,490.973990,143.549995,318.539690,115.0,495.0,143.0,325.0
1,JPEG_20161119_174038_1000690577600.png,25.396328,459.897020,62.030487,411.908624,14.900318,458.294285,57.182395,411.727916,23.0,457.0,61.0,409.0
2,147444927651111470309333776-Roadster-Men-Casua...,30.202265,593.225792,12.857371,463.697808,38.461993,584.657026,12.636195,458.999241,37.0,601.0,13.0,470.0
3,147772332675720161028_161611.png,24.748062,602.583612,161.349796,383.408718,27.214124,598.370536,164.150211,382.583902,27.0,602.0,162.0,385.0
4,1473315333651DeeplearnS11638.png,60.361998,586.745336,175.362128,327.451221,59.576765,587.784816,173.057213,328.764561,60.0,586.0,174.0,325.0


In [7]:
evalid = pd.read_csv('validation2.csv', index_col=0)
evalid.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160624_135055_1000527160000.png,132.316241,595.976432,173.387921,320.458870,140.286015,600.110038,173.996577,317.819759,132.0,606.0,165.0,325.0
1,147771890839520161028_141911.png,124.201832,549.382251,65.640745,459.768891,123.266852,543.815613,60.365218,453.204313,123.0,544.0,52.0,452.0
2,1474715198275DSC07675.png,305.480214,462.097976,25.053617,452.708393,304.909690,457.154105,31.033978,463.411200,311.0,458.0,32.0,459.0
3,JPEG_20161117_112207_1000880661419.png,130.057042,623.085676,53.449472,462.756976,0.000000,630.595921,0.000000,480.802100,13.0,640.0,5.0,459.0
4,147444941744811473248953580-Roadster-Men-Navy-...,36.165800,601.265808,28.245072,472.670098,36.505154,601.419939,20.901620,471.695088,34.0,603.0,16.0,471.0


In [11]:

x1a = Input(shape=(1,)) 
x2a = Input(shape=(1,))
y1a = Input(shape=(1,))
y2a = Input(shape=(1,))
x1b = Input(shape=(1,))
x2b = Input(shape=(1,))
y1b = Input(shape=(1,))
y2b = Input(shape=(1,))

x1 = Concatenate()([x1a, x1b])
x2 = Concatenate()([x2a, x2b])
y1 = Concatenate()([y1a, y1b])
y2 = Concatenate()([y2a, y2b])

x1 = Dense(1)(x1)
x2 = Dense(1)(x2)
y1 = Dense(1)(y1)
y2 = Dense(1)(y2)

model = Model(inputs=[x1a, x2a, y1a, y2a, x1b, x2b, y1b, y2b], outputs=[x1,x2,y1,y2])

model.summary()

model.compile(loss="mean_absolute_error", optimizer="adam")



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (I

In [12]:
stop = EarlyStopping(monitor="loss", patience=0, mode="max")

model.fit(np.transpose(etrain.iloc[:,1:9].values).tolist(), np.transpose(etrain.iloc[:,9:].values).tolist(), epochs=200, callbacks=[stop])



Epoch 1/200
14000/14000 [==============================] - 6s 455us/step - loss: 578.9359 - dense_loss: 47.3668 - dense_1_loss: 403.3397 - dense_2_loss: 123.0143 - dense_3_loss: 5.2150
Epoch 2/200
14000/14000 [==============================] - 3s 215us/step - loss: 96.8419 - dense_loss: 5.2622 - dense_1_loss: 40.3854 - dense_2_loss: 46.0047 - dense_3_loss: 5.1897
Epoch 3/200
14000/14000 [==============================] - 3s 196us/step - loss: 24.9920 - dense_loss: 5.2269 - dense_1_loss: 10.6604 - dense_2_loss: 3.9236 - dense_3_loss: 5.1810
Epoch 4/200
14000/14000 [==============================] - 3s 194us/step - loss: 24.6740 - dense_loss: 5.1752 - dense_1_loss: 10.6414 - dense_2_loss: 3.6990 - dense_3_loss: 5.1584
Epoch 5/200
14000/14000 [==============================] - 3s 201us/step - loss: 24.5846 - dense_loss: 5.1208 - dense_1_loss: 10.6305 - dense_2_loss: 3.6781 - dense_3_loss: 5.1552
Epoch 6/200
14000/14000 [==============================] - 3s 196us/step - loss: 24.4367 - den

14000/14000 [==============================] - 3s 203us/step - loss: 20.2088 - dense_loss: 3.6865 - dense_1_loss: 8.6123 - dense_2_loss: 3.2169 - dense_3_loss: 4.6931
Epoch 47/200
14000/14000 [==============================] - 3s 203us/step - loss: 20.1644 - dense_loss: 3.6838 - dense_1_loss: 8.5918 - dense_2_loss: 3.2146 - dense_3_loss: 4.6742
Epoch 48/200
14000/14000 [==============================] - 3s 201us/step - loss: 20.0852 - dense_loss: 3.6807 - dense_1_loss: 8.5247 - dense_2_loss: 3.2153 - dense_3_loss: 4.6645
Epoch 49/200
14000/14000 [==============================] - 3s 203us/step - loss: 20.0403 - dense_loss: 3.6821 - dense_1_loss: 8.4879 - dense_2_loss: 3.2095 - dense_3_loss: 4.6609
Epoch 50/200
14000/14000 [==============================] - 3s 199us/step - loss: 19.9756 - dense_loss: 3.6813 - dense_1_loss: 8.4233 - dense_2_loss: 3.2139 - dense_3_loss: 4.6571
Epoch 51/200
14000/14000 [==============================] - 3s 195us/step - loss: 19.9355 - dense_loss: 3.6776 - 

14000/14000 [==============================] - 3s 198us/step - loss: 18.1573 - dense_loss: 3.6717 - dense_1_loss: 6.8153 - dense_2_loss: 3.2101 - dense_3_loss: 4.4601
Epoch 92/200
14000/14000 [==============================] - 3s 197us/step - loss: 18.1663 - dense_loss: 3.6743 - dense_1_loss: 6.8078 - dense_2_loss: 3.2104 - dense_3_loss: 4.4738
Epoch 93/200
14000/14000 [==============================] - 3s 199us/step - loss: 18.1293 - dense_loss: 3.6763 - dense_1_loss: 6.7736 - dense_2_loss: 3.2152 - dense_3_loss: 4.4642
Epoch 94/200
14000/14000 [==============================] - 3s 200us/step - loss: 18.0971 - dense_loss: 3.6802 - dense_1_loss: 6.7349 - dense_2_loss: 3.2133 - dense_3_loss: 4.4687
Epoch 95/200
14000/14000 [==============================] - 3s 200us/step - loss: 18.0487 - dense_loss: 3.6700 - dense_1_loss: 6.7083 - dense_2_loss: 3.2142 - dense_3_loss: 4.4562
Epoch 96/200
14000/14000 [==============================] - 3s 201us/step - loss: 18.0235 - dense_loss: 3.6788 - 

14000/14000 [==============================] - 3s 198us/step - loss: 17.1209 - dense_loss: 3.6783 - dense_1_loss: 5.8186 - dense_2_loss: 3.2155 - dense_3_loss: 4.4084
Epoch 137/200
14000/14000 [==============================] - 3s 200us/step - loss: 17.0870 - dense_loss: 3.6714 - dense_1_loss: 5.7963 - dense_2_loss: 3.2126 - dense_3_loss: 4.4066
Epoch 138/200
14000/14000 [==============================] - 3s 200us/step - loss: 17.0674 - dense_loss: 3.6711 - dense_1_loss: 5.7944 - dense_2_loss: 3.2082 - dense_3_loss: 4.3938
Epoch 139/200
14000/14000 [==============================] - 3s 198us/step - loss: 17.0694 - dense_loss: 3.6759 - dense_1_loss: 5.7811 - dense_2_loss: 3.2152 - dense_3_loss: 4.3972
Epoch 140/200
14000/14000 [==============================] - 3s 200us/step - loss: 17.0381 - dense_loss: 3.6747 - dense_1_loss: 5.7509 - dense_2_loss: 3.2125 - dense_3_loss: 4.4000
Epoch 141/200
14000/14000 [==============================] - 3s 199us/step - loss: 17.0198 - dense_loss: 3.67

KeyboardInterrupt: 

In [13]:
preds = pd.DataFrame.from_records(model.predict(np.transpose(evalid.iloc[:,1:9].values).tolist()))
preds = preds.T
preds.head()

,0,1,2,3
0,[134.24901],[596.72626],[172.99782],[318.79565]
1,[125.16307],[544.2293],[65.14108],[455.78284]
2,[307.7944],[457.46387],[25.376484],[452.31262]
3,[117.827255],[625.8734],[49.891365],[463.74506]
4,[36.618748],[599.5562],[27.705736],[469.68005]


In [14]:
evalid.iloc[:,9:].head()

,9,10,11,12
0,132.0,606.0,165.0,325.0
1,123.0,544.0,52.0,452.0
2,311.0,458.0,32.0,459.0
3,13.0,640.0,5.0,459.0
4,34.0,603.0,16.0,471.0


In [15]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
#     assert bb1['x1'] < bb1['x2']
#     assert bb1['y1'] < bb1['y2']
#     assert bb2['x1'] < bb2['x2']
#     assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [16]:
iou = []

for index,row in evalid.iloc[:,1:].iterrows():
    
    bbox1 = {
        'x1':evalid.iloc[index, 1],
        'x2':evalid.iloc[index, 2],
        'y1':evalid.iloc[index, 3],
        'y2':evalid.iloc[index, 4]
    }
    
    bbox2 = {
        'x1':np.asscalar(preds.iloc[index, 0]),
        'x2':np.asscalar(preds.iloc[index, 1]),
        'y1':np.asscalar(preds.iloc[index, 2]),
        'y2':np.asscalar(preds.iloc[index, 3])
    }
    
    iou.append(get_iou(bbox1, bbox2))

np.mean(iou)

0.9740807363522546

In [17]:
test_df = pd.read_csv("test.csv")
test_df.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [22]:
df3 =  pd.DataFrame(np.zeros((len(test_df), 13)))

In [23]:
for index, row in test_df.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_mv(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    df3.iloc[index,0] = row['image_name']
    df3.iloc[index,1] = ((x1a))
    df3.iloc[index,2] = ((x2a))
    df3.iloc[index,3] = ((y1a))
    df3.iloc[index,4] = ((y2a))
    df3.iloc[index,5] = ((x1b))
    df3.iloc[index,6] = ((x2b))
    df3.iloc[index,7] = ((y1b))
    df3.iloc[index,8] = ((y2b))
    
    if (index%1000)==0:
        print(index)

df3.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1474723840903DSC08089.png,233.286459,442.107807,88.791412,408.305599,232.934222,443.137905,87.840108,405.383606,0.0,0.0,0.0,0.0
1,1473231475010DeeplearnS11276.png,75.543976,567.797476,142.480697,351.878813,73.057654,567.362409,136.734295,351.654543,0.0,0.0,0.0,0.0
2,JPEG_20161205_135307_1000155917326.png,146.332649,496.579026,54.420127,433.584295,147.640566,495.844334,55.824088,436.824908,0.0,0.0,0.0,0.0
3,JPEG_20160711_123440_1000518778437.png,212.218022,467.038305,92.153884,417.231298,213.478285,467.041452,91.416558,412.423750,0.0,0.0,0.0,0.0
4,JPEG_20160803_115329_100034020722.png,137.397624,498.003828,43.644762,426.797120,137.865928,497.681449,47.736981,433.489870,0.0,0.0,0.0,0.0


In [25]:
df3.to_csv('test_preds.csv')

In [26]:
preds = pd.DataFrame.from_records(model.predict(np.transpose(df3.iloc[:,1:9].values).tolist()))
preds = preds.T
preds.head()

,0,1,2,3
0,[235.09908],[441.18445],[88.511475],[405.6237]
1,[75.9931],[565.7768],[141.75992],[350.3631]
2,[147.68437],[494.4842],[54.376606],[431.87955]
3,[214.04301],[465.44373],[91.87931],[414.06808]
4,[138.59846],[496.16077],[43.800377],[425.869]


In [32]:
for index, row in preds.iterrows():

    test_df.iloc[index,1] = np.asscalar(row[0])
    test_df.iloc[index,2] = np.asscalar(row[1])
    test_df.iloc[index,3] = np.asscalar(row[2])
    test_df.iloc[index,4] = np.asscalar(row[3])

test_df.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,235.099,441.184,88.511475,405.623688
1,1473231475010DeeplearnS11276.png,75.9931,565.777,141.759918,350.363098
2,JPEG_20161205_135307_1000155917326.png,147.684,494.484,54.376606,431.879547
3,JPEG_20160711_123440_1000518778437.png,214.043,465.444,91.879311,414.068085
4,JPEG_20160803_115329_100034020722.png,138.598,496.161,43.800377,425.868988


In [34]:
test_df.to_csv('final_prediction.csv', index=False)